In [7]:
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/Scatter_Plot.ipynb
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/QQ_Plot.ipynb
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/Feature_Importance_Plot_Random_Forest_Regressor.ipynb

In [ ]:

def Random_Forest_Regression(X_train, X_test, y_train, y_test, filtered_data):
    # Function to be optimized by Bayesian Optimization
    def rf_regressor_cv(n_estimators, max_depth, min_samples_split, min_samples_leaf):
        # Ensure parameters are in the correct format
        n_estimators = int(n_estimators)
        max_depth = int(max_depth)
        min_samples_split = int(min_samples_split)
        min_samples_leaf = int(min_samples_leaf)

        # Define and fit the model
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )
        model.fit(X_train, y_train)

        # Predict and calculate R2 score
        predict_test = model.predict(X_test)
        r2_score_test = metrics.r2_score(y_test, predict_test)
        return r2_score_test

    # Setting ranges for hyperparameters
    pbounds = {
        'n_estimators': (10, 100),
        'max_depth': (5, 30),
        'min_samples_split': (2, 10),
        'min_samples_leaf': (1, 5)
    }

    # Bayesian Optimization
    optimizer = BayesianOptimization(
        f=rf_regressor_cv,
        pbounds=pbounds,
        random_state=0
    )
    optimizer.maximize(init_points=5, n_iter=15)

    # Optimal parameters
    params = optimizer.max['params']
    
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    params['min_samples_split'] = int(params['min_samples_split'])
    params['min_samples_leaf'] = int(params['min_samples_leaf'])
    
    print("Optimal Parameters:", params)

    # Create and train model with optimal parameters
    best_model = RandomForestRegressor(
        n_estimators = params['n_estimators'],
        max_depth = params['max_depth'],
        min_samples_split = params['min_samples_split'],
        min_samples_leaf = params['min_samples_leaf'],
        random_state=0
    )
    best_model.fit(X_train, y_train)

    # Predictions
    predict_train = best_model.predict(X_train)
    predict_test = best_model.predict(X_test)

    # DataFrame Creation
    train_df = pd.DataFrame({'Actual': y_train, 'Predicted': predict_train, 'Category': 'Training'}, index=y_train.index)
    test_df = pd.DataFrame({'Actual': y_test, 'Predicted': predict_test, 'Category': 'Testing'}, index=y_test.index)

    # Concatenating and Sorting
    combined_df = pd.concat([train_df, test_df])
    combined_df = combined_df.sort_index()

    # R2 Score
    r2_score_train = round(metrics.r2_score(y_train, predict_train), 2)
    r2_score_test = round(metrics.r2_score(y_test, predict_test), 2)
    print('R2 Score Train:', r2_score_train)
    print('R2 Score Test:', r2_score_test)

    return best_model, params, r2_score_test

In [6]:
# def Random_Forest_Regression(X_train, X_test, y_train, y_test,filtered_data):
#     # Define the objective function for Bayesian optimization
#     def objective(params):
#         n_estimators, max_depth, min_samples_split, min_samples_leaf = params

#         model = RandomForestRegressor(n_estimators=n_estimators, 
#                                       max_depth=max_depth, 
#                                       min_samples_split=min_samples_split, 
#                                       min_samples_leaf=min_samples_leaf,
#                                       random_state=0)
#         model.fit(X_train, y_train)
#         predict_train = model.predict(X_train)
#         r2 = r2_score(y_train, predict_train)

#         return -r2

#     # Define the hyperparameter space
#     param_space = [
#         Integer(10, 100, name="n_estimators"),
#         Integer(1, 30, name="max_depth"),
#         Integer(2, 10, name="min_samples_split"),
#         Integer(1, 10, name="min_samples_leaf")
#     ]

#     # Perform Bayesian optimization
#     results = gp_minimize(objective, param_space, n_calls=50, random_state=0)

#     # Extract the best hyperparameters
#     n_estimators, max_depth, min_samples_split, min_samples_leaf = results.x
    

#     print("target: ",v)
#     print("number of esimators: ",n_estimators) 
#     print("maximum depth: ", max_depth)
#     print("minimum_samples_split: ",min_samples_split) 
#     print("min_samples_leaf: ",min_samples_leaf)


#     # Create and train the model with the best hyperparameters
#     best_model = RandomForestRegressor(n_estimators=n_estimators, 
#                                   max_depth=max_depth, 
#                                   min_samples_split=min_samples_split, 
#                                   min_samples_leaf=min_samples_leaf, 
#                                   random_state=0)
#     best_model.fit(X_train, y_train)

#     # Make predictions
#     predict_train = best_model.predict(X_train)
#     predict_test = best_model.predict(X_test)

#     # Create DataFrames with indices and categories
#     train_df = pd.DataFrame({'Actual': y_train, 'Predicted': predict_train, 'Category': 'Training'}, index=y_train.index)
#     test_df = pd.DataFrame({'Actual': y_test, 'Predicted': predict_test, 'Category': 'Testing'}, index=y_test.index)

#     # Concatenate the DataFrames
#     combined_df = pd.concat([train_df, test_df])
#     combined_df = combined_df.sort_index()
#     print(combined_df)

#     # Calculate R2 values
#     r2_score_train = round(r2_score(y_train, predict_train), 2)
#     r2_score_test = round(r2_score(y_test, predict_test), 2)
#     print("R2 value of train data:", r2_score_train)
#     print("R2 value of test data:", r2_score_test)
    
#     if (v == 'dCn'):
#         unit = 'dCn'
#     else:
#         unit = 'pm_conc'

#     Scatter_Plot(combined_df, train_df, test_df, r2_score_train, r2_score_test, v, unit)
#     qq_plot(test_df, v, unit)
#     Feature_Importance_Random_Forest_Regressor(best_model, filtered_data, v, unit)
#     return best_model,results.x, r2_score_test
#     # #-------Time Series for Random Forest Regression ------#
#     # fig, ax = plt.subplots()
#     # y_test = pd.DataFrame(y_test.to_numpy())
#     # predict_test = pd.DataFrame(predict_test)
#     # time_test  = pd.DataFrame(X_test["dateTime"].to_numpy())
#     # df_timeseries_test = pd.concat([time_test,y_test],axis = 1)
#     # df_timeseries_predict = pd.concat([time_test,predict_test],axis = 1)
#     # df_timeseries_test.columns =["dateTime","test"]
#     # df_timeseries_predict.columns =["dateTime","predict"]
#     # df_timeseries_test = df_timeseries_test.sort_values(by='dateTime',ascending=True)
#     # df_timeseries_predict = df_timeseries_predict.sort_values(by='dateTime',ascending=True)
#     # ax.plot(df_timeseries_test.iloc[:,0],df_timeseries_test.iloc[:,1], linewidth = 2, label ="Actual",color = "red")
#     # ax.plot(df_timeseries_predict.iloc[:,0],df_timeseries_predict.iloc[:,1], linewidth = 2,label = "Estimated", color = "green")
#     # plt.xlabel("Date")
#     # plt.ylabel(y_train.name)
#     # plt.title("Time Series for Random Forest Regression")
#     # myFmt = DateFormatter('%Y-%m-%d')
#     # ax.xaxis.set_major_formatter(myFmt)
    
#     ## Rotate date labels automatically
#     # fig.autofmt_xdate()
#     # plt.legend(loc="upper left")
#     # plt.show()

    
#     # Evaluating the Algorithm
#     #print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predict_test))  
#     #print('Mean Squared Error:', metrics.mean_squared_error(y_test, predict_test))  
#     #print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predict_test)))
#     # label = y_test.name[:-len("_grimm")]
     
